In [ ]:
pip install git+https://github.com/microsoft/recommenders.git

  Cloning https://github.com/microsoft/recommenders.git to /tmp/pip-req-build-h3zjlwko
  Running command git clone -q https://github.com/microsoft/recommenders.git /tmp/pip-req-build-h3zjlwko
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 61 kB 7.4 MB/s 
     |████████████████████████████████| 2.8 MB 40.9 MB/s 
     |████████████████████████████████| 1.5 MB 39.0 MB/s 
     |████████████████████████████████| 636 kB 34.8 MB/s 
     |████████████████████████████████| 310 kB 44.6 MB/s 
     |████████████████████████████████| 11.8 MB 51 kB/s 
     |████████████████████████████████| 59 kB 7.0 MB/s 
     |████████████████████████████████| 93 kB 2.3 MB/s 
     |████████████████████████████████| 12.4 MB 33 kB/s 
     |████████████████████████████████| 895 kB 39.9 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 3.3 MB 32.6 MB

In [ ]:
pip install pyspark

     |████████████████████████████████| 212.4 MB 55 kB/s 
     |████████████████████████████████| 198 kB 50.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=f5278b3558c23a8801533ea6591239c3864ba1f9f7b8b247c179d76380ccff1b
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


### Global Setup

In [ ]:
import numpy as np
import pandas as pd
import recommenders

import sys
import pyspark
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, FloatType, IntegerType, LongType

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.spark_splitters import spark_random_split
from recommenders.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation
from recommenders.utils.spark_utils import start_or_get_spark

print("System version: {}".format(sys.version))
print("Spark version: {}".format(pyspark.__version__))

System version: 3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
Spark version: 3.1.2


Getting Data from drive. Not critical to overall project, but a helpful if running only on colab. Otherwise, local files are ok

In [ ]:
import gdown
activity_raw_url = 'https://drive.google.com/file/d/1GZ7Pg06hoMGdfDtcAPPNPnQ_ZwIFcEh7/view?usp=sharing'
activity_url     ='https://drive.google.com/uc?id=' + activity_raw_url.split('/')[-2] #Do not change
gdown.download(activity_url, 'activity.csv',quiet=False)

student_info_raw_url = 'https://drive.google.com/file/d/1Y0EmJMalLuoSSe4_CHzf7aSWBV4BEcH4/view?usp=sharing'
student_info_url     ='https://drive.google.com/uc?id=' + student_info_raw_url.split('/')[-2] #Do not change
gdown.download(student_info_url, 'student_info.csv',quiet=False)

student_activity_train_raw_url = 'https://drive.google.com/file/d/1xaXuFboFYe4fwxUBr9ygWlcfaIX-cTbc/view?usp=sharing'
student_activity_train_url     ='https://drive.google.com/uc?id=' + student_activity_train_raw_url.split('/')[-2] #Do not change
gdown.download(student_activity_train_url, 'student_activity_train.csv',quiet=False)

student_activity_test_raw_url = 'https://drive.google.com/file/d/17Rd0DTaQ4APmD0L4sZTHJWMhfdflI5N7/view?usp=sharing'
student_activity_test_url     ='https://drive.google.com/uc?id=' + student_activity_test_raw_url.split('/')[-2] #Do not change
gdown.download(student_activity_test_url, 'student_activity_test.csv',quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1GZ7Pg06hoMGdfDtcAPPNPnQ_ZwIFcEh7
To: /content/activity.csv
100%|██████████| 18.9k/18.9k [00:00<00:00, 9.93MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y0EmJMalLuoSSe4_CHzf7aSWBV4BEcH4
To: /content/student_info.csv
100%|██████████| 24.9k/24.9k [00:00<00:00, 33.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1xaXuFboFYe4fwxUBr9ygWlcfaIX-cTbc
To: /content/student_activity_train.csv
100%|██████████| 1.06M/1.06M [00:00<00:00, 24.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=17Rd0DTaQ4APmD0L4sZTHJWMhfdflI5N7
To: /content/student_activity_test.csv
100%|██████████| 222k/222k [00:00<00:00, 10.4MB/s]


'student_activity_test.csv'

### Data Exploration

In [ ]:
df_activity = pd.read_csv("activity.csv", index_col=0)
print(df_activity.shape)
df_activity.head()

(155, 25)


,Activity_ID,Activity_Name,Activity_Content,Activity_Type,GameType,Ages_2yo,Ages_3yo,Ages_4yo,Ages_5yo,Ages_6yo,Ages_7yo,Cultures_None,Cultures_Brazil,Cultures_US,Cultures_UK,Cultures_Denmark,Cultures_Spain,Cultures_Mexico,Cultures_Canada,Cultures_Portugal,Cultures_Australia,Cultures_France,Cultures_Italy,Cultures_Germany,Activity_Level
0,farm-poiki-book-1,"Book: Poiki, the farmer",farm,standard,reading-book,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,farm-poiki-book-2,Book: The animals in the farm,farm-animals,standard,reading-book,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,farm-wendy-book-1,Book: Colors in the farm,colors,standard,reading-book,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,farm-kevin-book-1,Book: Shapes in the farm,shapes,standard,reading-book,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,country-none-book-1,How are the farms in my country?,my-country,cultural,reading-book,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,0,0,0,0


In [ ]:
df_students = pd.read_csv("student_info.csv", index_col=0)
print(df_students.shape)
df_students.head()

(250, 13)


,User_ID,Gender,User_Name,Age,Translanguage_Level,Literate,Learning_Rate,Country_of_Birth,Country_of_Residence,Cultural_Heritage,Main_Language,Learning_Language,Learning_Profile
0,0,F,Norma Fisher,5,2,Complete,Slow,Denmark,Canada,"['Portugal', 'Australia']",Portuguese,English,HL
1,1,F,Kayla Sullivan,6,1,Not_Started,Fast,UK,Canada,['France'],French,English,BTB
2,2,F,Elizabeth Woods,6,2,Not_Started,Slow,Brazil,Canada,['Portugal'],Portuguese,French,IL
3,3,F,Susan Wagner,4,3,Complete,Fast,UK,Portugal,"['France', 'Canada']",French,English,HL
4,4,M,Peter Montgomery,3,2,Complete,Fast,France,Brazil,"['Germany', 'Spain', 'Italy']",German,Spanish,HL


In [ ]:
df_scores_train = pd.read_csv("student_activity_train.csv", index_col=0)
print(df_scores_train.shape)
df_scores_train.head()

(11880, 7)


,User_ID,Activity_ID,No_of_attempts,Time_taken,Translanguage_Level,Usefulness_Score,_Translanguage_Level_Before_Course_Plan
0,0,country-wendy-videoclip-1,3,278.055180,3.101002,0.139800,2
1,0,country-yasmin-videoclip-1,4,281.867673,2.826381,0.084624,2
2,0,lab-wendy-colors-1,3,277.465027,2.782403,0.065831,2
3,0,who-wendy-colors-1,2,283.909252,3.006378,0.011640,2
4,0,farm-poiki-video-1,3,285.101000,2.736871,0.038446,2


In [ ]:
df_scores_test = pd.read_csv("student_activity_test.csv", index_col=0)
print(df_scores_test.shape)
df_scores_test.head()

(2500, 7)


,User_ID,Activity_ID,No_of_attempts,Time_taken,Translanguage_Level,Usefulness_Score,_Translanguage_Level_Before_Course_Plan
0,0,cooking-yasmin-simulation-1,4,276.452842,3.487373,0.390101,3
1,0,farm-poiki-video-2,1,292.072599,3.711785,0.168906,3
2,0,lab-kevin-game-1,1,280.767327,3.860060,0.092441,3
3,0,insects-roulette-1,2,286.974125,3.731912,0.079838,3
4,0,cooking-yasmin-simulation-1,3,286.916326,3.823462,0.055227,3


### Build a numerical activity ID as the ALS currently only supports integers for user and item ids.

### Build a mapping Activity_ID -> encoded (numerical) activity ID

In [ ]:
# make sure `Activity_ID` in `df_activity` is unique
assert len(df_activity) == len(df_activity["Activity_ID"].unique())

activity_id_to_encoded_id_dict = df_activity["Activity_ID"].to_dict()
print(activity_id_to_encoded_id_dict)
1encoded_id_to_activity_dict = {v: k for k, v in activity_id_to_encoded_id_dict.items()}  #inverted
print(encoded_id_to_activity_dict)

{0: 'farm-poiki-book-1', 1: 'farm-poiki-book-2', 2: 'farm-wendy-book-1', 3: 'farm-kevin-book-1', 4: 'country-none-book-1', 5: 'country-none-book-2', 6: 'country-none-book-3', 7: 'country-none-book-4', 8: 'home-kevin-book-1', 9: 'daycare-all-book-1', 10: 'world-all-book-1', 11: 'home-wendy-book-1', 12: 'travel-yasmin-book-1', 13: 'who-wendy-book-1', 14: 'who-wendy-book-2', 15: 'lab-wendy-book-1', 16: 'farm-poiki-videoclip-1', 17: 'farm-poiki-videoclip-2', 18: 'theatre-all-videoclip-1', 19: 'country-yasmin-videoclip-1', 20: 'country-wendy-videoclip-1', 21: 'country-yasmin-videoclip-2', 22: 'who-all-videoclip-1', 23: 'farm-yasmin-demovideo-1', 24: 'farm-poiki-demovideo-1', 25: 'farm-kevin-demovideo-1', 26: 'farm-wendy-demovideo-1', 27: 'farm-kevin-demovideo-2', 28: 'farm-poiki-demovideo-2', 29: 'farm-poiki-demovideo-3', 30: 'farm-none-demovideo-1', 31: 'farm-poiki-demovideo-4', 32: 'lab-wendy-demovideo-1', 33: 'who-wendy-demovideo-1', 34: 'who-yasmin-demovideo-1', 35: 'who-kevin-demovideo

### Add the encoded (numerical) activity id to `df_activitiy`

In [ ]:
df_activity['Activity_Num_ID'] = df_activity['Activity_ID'].map(encoded_id_to_activity_dict)
df_activity.head()

,Activity_ID,Activity_Name,Activity_Content,Activity_Type,GameType,Ages_2yo,Ages_3yo,Ages_4yo,Ages_5yo,Ages_6yo,Ages_7yo,Cultures_None,Cultures_Brazil,Cultures_US,Cultures_UK,Cultures_Denmark,Cultures_Spain,Cultures_Mexico,Cultures_Canada,Cultures_Portugal,Cultures_Australia,Cultures_France,Cultures_Italy,Cultures_Germany,Activity_Level,Activity_Num_ID
0,farm-poiki-book-1,"Book: Poiki, the farmer",farm,standard,reading-book,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,farm-poiki-book-2,Book: The animals in the farm,farm-animals,standard,reading-book,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,farm-wendy-book-1,Book: Colors in the farm,colors,standard,reading-book,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,farm-kevin-book-1,Book: Shapes in the farm,shapes,standard,reading-book,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3
4,country-none-book-1,How are the farms in my country?,my-country,cultural,reading-book,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,0,0,0,0,4


### Add the encoded (numerical) activity id to `df_scores_train`

In [ ]:
df_scores_train['Activity_Num_ID'] = df_scores_train['Activity_ID'].map(encoded_id_to_activity_dict)
df_scores_train.head()

,User_ID,Activity_ID,No_of_attempts,Time_taken,Translanguage_Level,Usefulness_Score,_Translanguage_Level_Before_Course_Plan,Activity_Num_ID
0,0,country-wendy-videoclip-1,3,278.055180,3.101002,0.139800,2,20
1,0,country-yasmin-videoclip-1,4,281.867673,2.826381,0.084624,2,19
2,0,lab-wendy-colors-1,3,277.465027,2.782403,0.065831,2,110
3,0,who-wendy-colors-1,2,283.909252,3.006378,0.011640,2,135
4,0,farm-poiki-video-1,3,285.101000,2.736871,0.038446,2,66


### Add the encoded (numerical) activity id to `df_scores_test`

In [ ]:
df_scores_test['Activity_Num_ID'] = df_scores_test['Activity_ID'].map(encoded_id_to_activity_dict)
df_scores_test.head()

,User_ID,Activity_ID,No_of_attempts,Time_taken,Translanguage_Level,Usefulness_Score,_Translanguage_Level_Before_Course_Plan,Activity_Num_ID
0,0,cooking-yasmin-simulation-1,4,276.452842,3.487373,0.390101,3,102
1,0,farm-poiki-video-2,1,292.072599,3.711785,0.168906,3,67
2,0,lab-kevin-game-1,1,280.767327,3.860060,0.092441,3,136
3,0,insects-roulette-1,2,286.974125,3.731912,0.079838,3,146
4,0,cooking-yasmin-simulation-1,3,286.916326,3.823462,0.055227,3,102


### Set up Spark context

In [ ]:
# the following settings work well for debugging locally on VM - change when running on a cluster
# set up a giant single executor with many threads and specify memory cap
spark = start_or_get_spark("ALS PySpark", memory="16g")

In [ ]:
# top k items to recommend
TOP_K = 10

### Create the train and test dataset using spark

In [ ]:
# Note: The DataFrame-based API for ALS currently only supports integers for user and item ids.
schema = StructType(
    (
        StructField("User_ID", IntegerType()),
        StructField("Activity_Num_ID", IntegerType()),
        StructField("Usefulness_Score", FloatType()),
        # StructField("Timestamp", LongType()),
    )
)


In [ ]:
train = spark.createDataFrame(df_scores_train[['User_ID', 'Activity_Num_ID', 'Usefulness_Score']], schema=schema)
train.show()

+-------+---------------+----------------+
|User_ID|Activity_Num_ID|Usefulness_Score|
+-------+---------------+----------------+
|      0|             20|      0.13979952|
|      0|             19|      0.08462363|
|      0|            110|      0.06583124|
|      0|            135|     0.011639966|
|      0|             66|      0.03844588|
|      0|            140|      0.12760982|
|      0|              4|      0.38467258|
|      0|             67|       0.2677407|
|      0|            142|      0.23321155|
|      0|              6|      0.11221865|
|      0|            144|      0.49340522|
|      0|            102|      0.18139784|
|      0|            146|      0.23465453|
|      0|            140|      0.08161085|
|      0|            102|       0.1039794|
|      0|             14|     0.042830396|
|      0|             66|     0.043932125|
|      0|            119|     0.023190195|
|      0|             67|            -0.0|
|      0|            136|     9.232206E-4|
+-------+--

In [ ]:
test = spark.createDataFrame(df_scores_test[['User_ID', 'Activity_Num_ID', 'Usefulness_Score']], schema=schema)
test.show()

+-------+---------------+----------------+
|User_ID|Activity_Num_ID|Usefulness_Score|
+-------+---------------+----------------+
|      0|            102|      0.39010146|
|      0|             67|      0.16890566|
|      0|            136|      0.09244091|
|      0|            146|     0.079837985|
|      0|            102|      0.05522651|
|      0|             14|       0.5059616|
|      0|             66|      0.11612179|
|      0|            140|     0.096445695|
|      0|            144|      0.26811612|
|      0|            119|      0.12221528|
|      1|            140|      0.55767816|
|      1|            135|      0.46363315|
|      1|            145|      0.21864375|
|      1|            147|       0.5606937|
|      1|             14|      0.47170857|
|      1|            137|       0.5093477|
|      1|             67|       0.4523332|
|      1|             15|       0.4884733|
|      1|            119|      0.45285666|
|      1|            136|      0.48469877|
+-------+--

### Train the ALS model on the training data, and get the top-k recommendations for our testing data

In [ ]:
header = {
    "userCol": "User_ID",
    "itemCol": "Activity_Num_ID",
    "ratingCol": "Usefulness_Score",
}


als = ALS(
    rank=10,
    maxIter=15,
    implicitPrefs=False,
    regParam=0.05,
    coldStartStrategy='drop',
    nonnegative=False,
    seed=42,
    **header
)

In [ ]:
with Timer() as train_time:
    model = als.fit(train)

print("Took {} seconds for training.".format(train_time.interval))

Took 12.719900020000011 seconds for training.


In [ ]:
with Timer() as test_time:

    # Get the cross join of all user-item pairs and score them.
    users = train.select('User_ID').distinct()
    items = train.select('Activity_Num_ID').distinct()
    user_item = users.crossJoin(items)
    dfs_pred = model.transform(user_item)

    # Remove seen items.
    dfs_pred_exclude_train = dfs_pred.alias("pred").join(
        train.alias("train"),
        (dfs_pred['User_ID'] == train['User_ID']) & (dfs_pred['Activity_Num_ID'] == train['Activity_Num_ID']),
        how='outer'
    )

    top_all = dfs_pred_exclude_train.filter(dfs_pred_exclude_train["train.Usefulness_Score"].isNull()) \
        .select('pred.' + 'User_ID', 'pred.' + 'Activity_Num_ID', 'pred.' + "prediction")

    # In Spark, transformations are lazy evaluation
    # Use an action to force execute and measure the test time 
    top_all.cache().count()

print("Took {} seconds for prediction.".format(test_time.interval))

Took 259.79947379699996 seconds for prediction.


In [ ]:
top_all.show()

+-------+---------------+-----------+
|User_ID|Activity_Num_ID| prediction|
+-------+---------------+-----------+
|      3|             22| 0.20239624|
|      3|             89| 0.15910716|
|     12|            115| 0.12566525|
|     17|            138| 0.22016704|
|     18|             68| 0.47474045|
|     20|            121|  0.1408331|
|     21|            140|   0.251435|
|     25|             61|   0.451927|
|     27|             65|  0.5214742|
|     28|             63| 0.33487332|
|     32|             26| 0.37788752|
|     35|             45|  0.5765959|
|     37|            138| 0.22469497|
|     43|            100| 0.49194843|
|     56|             26| 0.11153958|
|     58|             34|0.048314597|
|     58|             81|0.096042395|
|     58|             93| 0.06463966|
|     60|             84| 0.40883777|
|     61|             13|  0.2476313|
+-------+---------------+-----------+
only showing top 20 rows



### Evaluate how well ALS performs

In [ ]:
rank_eval = SparkRankingEvaluation(test, top_all, k = TOP_K, col_user="User_ID", col_item="Activity_Num_ID", 
                                    col_rating="Usefulness_Score", col_prediction="prediction", 
                                    relevancy_method="top_k")

In [ ]:
print("Model:\tALS",
      "Top K:\t%d" % rank_eval.k,
      "MAP:\t%f" % rank_eval.map_at_k(),
      "NDCG:\t%f" % rank_eval.ndcg_at_k(),
      "Precision@K:\t%f" % rank_eval.precision_at_k(),
      "Recall@K:\t%f" % rank_eval.recall_at_k(), sep='\n')

Model:	ALS
Top K:	10
MAP:	0.000050
NDCG:	0.000278
Precision@K:	0.000400
Recall@K:	0.000400


### Evaluate rating prediction

In [ ]:
# Generate predicted ratings.
prediction = model.transform(test)
prediction.cache().show()

+-------+---------------+----------------+----------+
|User_ID|Activity_Num_ID|Usefulness_Score|prediction|
+-------+---------------+----------------+----------+
|     27|            137|       0.6228585| 0.5683333|
|    192|            137|      0.17410272|0.54196334|
|      1|            137|       0.5093477| 0.6442136|
|    117|            137|     0.039939724| 0.1382233|
|    179|            137|      0.11204991|0.21670061|
|     17|            137|      0.23561443| 0.3843096|
|     55|            137|       0.4837254| 0.3977306|
|    181|            137|      0.40124723|0.22585054|
|    157|             65|      0.12371699|0.28512174|
|     47|             65|      0.11421632|0.26712978|
|     54|             65|      0.27657962| 0.6711551|
|     87|             65|      0.01716716| 0.4087825|
|    141|             65|       0.4039578|0.10350898|
|    145|             65|       0.5833708| 0.6785665|
|    188|             65|       0.3424083|0.73829913|
|    118|             65|   

In [ ]:
rating_eval = SparkRatingEvaluation(test, prediction, col_user="User_ID", col_item="Activity_Num_ID", 
                                    col_rating="Usefulness_Score", col_prediction="prediction")

print("Model:\tALS rating prediction",
      "RMSE:\t%f" % rating_eval.rmse(),
      "MAE:\t%f" % rating_eval.mae(),
      "Explained variance:\t%f" % rating_eval.exp_var(),
      "R squared:\t%f" % rating_eval.rsquared(), sep='\n')

Model:	ALS rating prediction
RMSE:	0.263660
MAE:	0.189872
Explained variance:	-0.579172
R squared:	-0.825282


### Show predictions in pandas

In [ ]:
# prediction for user 0
resultUser0 = top_all[top_all['User_ID'] == 0]

In [ ]:
resultUser0 = resultUser0.sort(resultUser0.prediction.desc())

In [ ]:
resultUser0.show()

+-------+---------------+----------+
|User_ID|Activity_Num_ID|prediction|
+-------+---------------+----------+
|      0|             44|0.20318745|
|      0|            100| 0.1760679|
|      0|             78| 0.1742489|
|      0|             35|0.17127803|
|      0|             43| 0.1711221|
|      0|             13|0.16606975|
|      0|            126|0.16470516|
|      0|            153|0.16429436|
|      0|            115|0.16416253|
|      0|            143|0.16398223|
|      0|             45|0.16283046|
|      0|             21|0.16138808|
|      0|            147|0.15896507|
|      0|             22|0.15882505|
|      0|             26|0.15817665|
|      0|            154|0.15670955|
|      0|            118| 0.1562823|
|      0|            128|0.14911705|
|      0|             34|0.14782806|
|      0|            152| 0.1471505|
+-------+---------------+----------+
only showing top 20 rows



In [ ]:
df_resultUser0 = resultUser0.select("*").toPandas()

In [ ]:
df_resultUser0.head(n=10)

,User_ID,Activity_Num_ID,prediction
0,0,44,0.203187
1,0,100,0.176068
2,0,78,0.174249
3,0,35,0.171278
4,0,43,0.171122
5,0,13,0.166070
6,0,126,0.164705
7,0,153,0.164294
8,0,115,0.164163
9,0,143,0.163982


In [ ]:
# merge top 10 prediction with activities dataframe
df_resultUser0 = pd.merge(df_resultUser0, df_activity, on='Activity_Num_ID', how='left')

In [ ]:
print("User 0 top 10 predictions")
df_resultUser0[['prediction', 'User_ID', 'Activity_Num_ID', 'Activity_Name']].head(n=10)

User 0 top 10 predictions


,prediction,User_ID,Activity_Num_ID,Activity_Name
0,0.203187,0,44,Flashcards - fruits
1,0.176068,0,100,Preparing some food from my country
2,0.174249,0,78,6-pieces puzzle: Yasmin in my country
3,0.171278,0,35,Demo vídeo: decorating the cake
4,0.171122,0,43,Flashcards - musical instruments
5,0.166070,0,13,Book: Wendy is a scientist
6,0.164705,0,126,My name
7,0.164294,0,153,4 pairs memory game
8,0.164163,0,115,Poiki is harvesting with his tractor
9,0.163982,0,143,Roulette - get the right clothes
